# 0. Imports

In [8]:
# imports
import pandas as pd
import numpy as np
import datetime as dt
import time
import os
import pickle

from colorama import Fore, Style
from sklearn.pipeline import Pipeline
from joblib import dump, load

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Importing those functions from the API package so they can be used
# by the loaded pipeline in the API package later

from decipherer.ml_logic.encoders import ffill_nan, add_datetime_features

In [9]:
# environment variables
LOCAL_REGISTRY_PATH = "../training_outputs"

# prerequisite: having a "training_outputs" folder at the root of the project
! tree ../training_outputs

../training_outputs
├── metrics
├── models
└── params

3 directories, 0 files


# 1. Build pipeline

In [10]:
# Get rid of nan values using ffil
# def ffill_nan(X):
#     return X.fillna(method='ffill', axis = 0)

In [11]:
# # Create new datetime features
# def add_datetime_features(X):
    
#     # Copy X to avoid pandas warning
#     X_rep = X.copy()
    
#     # Handle datetime format
#     datetime = pd.to_datetime(X_rep['date'] + ' ' + X_rep['time'])
    
#     # Create new features using month, weekday, hour and minute
#     X_rep['month'] = datetime.dt.month
#     X_rep['weekday'] = datetime.dt.weekday
#     X_rep['hour'] = datetime.dt.hour
#     X_rep['minute'] = datetime.dt.minute
    
#     # Consider periodic effects
#     X_rep['month_sin'] = np.sin(2*np.pi*X_rep['month']/12)
#     X_rep['month_cos'] = np.cos(2*np.pi*X_rep['month']/12)
    
#     # Get rid of Datetime
#     return X_rep.drop(columns=['date', 'time'])

In [12]:
# Create a pipeline to preprocess the data

n_estimators = 10

features = ['global_active_power', 'global_reactive_power', 'voltage', 'global_intensity', 'global_consumption']
datetimes = ['date', 'time']

preparator = ColumnTransformer([ 
    ('imputer', FunctionTransformer(ffill_nan), features),
    ('datetime_features_adder', FunctionTransformer(add_datetime_features), datetimes)
])

pipeline = Pipeline([
    ('preparator', preparator),
    ('std_scaler', StandardScaler()),
    ('estimator', RandomForestRegressor(n_estimators=n_estimators))
])

pipeline

Pipeline(steps=[('preparator',
                 ColumnTransformer(transformers=[('imputer',
                                                  FunctionTransformer(func=<function ffill_nan at 0x13e97f790>),
                                                  ['global_active_power',
                                                   'global_reactive_power',
                                                   'voltage',
                                                   'global_intensity',
                                                   'global_consumption']),
                                                 ('datetime_features_adder',
                                                  FunctionTransformer(func=<function add_datetime_features at 0x13e97f820>),
                                                  ['date', 'time'])])),
                ('std_scaler', StandardScaler()),
                ('estimator', RandomForestRegressor(n_estimators=10))])

# 3. Prepare the data to train

In [13]:
# Take just a subset of the data for now (year==2008)
data = pd.read_csv('../data/household_power_consumption.txt', sep=';', na_values='?')
data = data[data.Date.str.endswith('2008')]
data.columns = data.columns.str.lower()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 527040 entries, 547596 to 1074635
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   date                   527040 non-null  object 
 1   time                   527040 non-null  object 
 2   global_active_power    526905 non-null  float64
 3   global_reactive_power  526905 non-null  float64
 4   voltage                526905 non-null  float64
 5   global_intensity       526905 non-null  float64
 6   sub_metering_1         526905 non-null  float64
 7   sub_metering_2         526905 non-null  float64
 8   sub_metering_3         526905 non-null  float64
dtypes: float64(7), object(2)
memory usage: 40.2+ MB


In [14]:
labels = ['sub_metering_1', 'sub_metering_2', 'sub_metering_3']
data['global_consumption'] = data[labels].sum(axis=1)

In [15]:
# Create X, y and save datetime in a separeted column
X = data.drop(columns=labels)
y = data[labels]
X.shape, y.shape

((527040, 7), (527040, 3))

In [16]:
y.isna().sum()

sub_metering_1    135
sub_metering_2    135
sub_metering_3    135
dtype: int64

In [17]:
# Remove na values from y
y = y.fillna(method='ffill', axis=0)

In [18]:
X.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
547596,1/1/2008,00:00:00,1.620,0.070,241.25,6.6,18.0
547597,1/1/2008,00:01:00,1.626,0.072,241.74,6.6,18.0
547598,1/1/2008,00:02:00,1.622,0.072,241.52,6.6,18.0
547599,1/1/2008,00:03:00,1.612,0.070,240.82,6.6,18.0
547600,1/1/2008,00:04:00,1.612,0.070,240.80,6.6,18.0


In [19]:
y.head()

,sub_metering_1,sub_metering_2,sub_metering_3
547596,0.0,0.0,18.0
547597,0.0,0.0,18.0
547598,0.0,0.0,18.0
547599,0.0,0.0,18.0
547600,0.0,0.0,18.0


In [20]:
# Train/test Split /!\ For later, if we use sequential models (ARIMA, RNN): see if we consider a TS special train/test split (to keep sequence's logic)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((368928, 7), (158112, 7), (368928, 3), (158112, 3))

In [21]:
X_train.head()

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
707089,20/4/2008,18:13:00,0.468,0.212,241.14,2.0,1.0
577145,21/1/2008,12:29:00,1.454,0.080,237.32,6.2,18.0
548121,1/1/2008,08:45:00,1.536,0.334,241.90,6.4,1.0
669244,25/3/2008,11:28:00,1.362,0.072,241.09,5.6,20.0
890338,26/8/2008,00:22:00,0.186,0.148,239.48,1.0,1.0


# 4. Train and save the pipeline

In [22]:
# Function to save the pipeline locally
def save_pipeline(pipeline: Pipeline = None,
                  params: dict = None,
                  metrics: dict = None) -> None:
    """
    persist trained pipeline, params and metrics
    """

    timestamp = time.strftime("%Y%m%d-%H%M%S")

    print(Fore.BLUE + "\nSave pipeline to local disk..." + Style.RESET_ALL)

    # save params
    if params is not None:
        params_path = os.path.join(LOCAL_REGISTRY_PATH, "params", timestamp + ".pickle")
        with open(params_path, "wb") as file:
            pickle.dump(params, file)

    # save metrics
    if metrics is not None:
        metrics_path = os.path.join(LOCAL_REGISTRY_PATH, "metrics", timestamp + ".pickle")
        with open(metrics_path, "wb") as file:
            pickle.dump(metrics, file)

    # save pipeline
    if pipeline is not None:
        pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", timestamp + ".joblib")
        dump(pipeline, pipeline_path)
         
    print("\n✅ data saved locally")

    return None

In [23]:
%%time

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Mesure its performance
r2_score = pipeline.score(X_test, y_test)
#r2_score = pipeline.score(X_test[['Voltage']].dropna(), y_test.dropna())

# Save it locally
params = dict(
    # Model parameters
    n_estimators=10,

    # Package behavior
    context="train",

    # Data used to fit
    dataset_start=data.date.iloc[0],
    dataset_end=data.date.iloc[-1]
)

metrics = dict(r2_score=r2_score)
print(metrics)

save_pipeline(pipeline, params, metrics)

{'r2_score': 0.8910974311089367}

Save pipeline to local disk...

✅ data saved locally
CPU times: user 1min 2s, sys: 307 ms, total: 1min 3s
Wall time: 1min 3s


# 5. Load the pipeline and try to predict

In [24]:
pipeline_path = os.path.join(LOCAL_REGISTRY_PATH, "models", "20221123-170451.joblib")
pipeline_loaded = load(pipeline_path)
pipeline_loaded

FileNotFoundError: [Errno 2] No such file or directory: '../training_outputs/models/20221123-170451.joblib'

In [ ]:
X_test

,date,time,global_active_power,global_reactive_power,voltage,global_intensity,global_consumption
880888,19/8/2008,10:52:00,0.324,0.368,238.96,2.0,2.0
657473,17/3/2008,07:17:00,2.966,0.078,240.30,12.4,20.0
713866,25/4/2008,11:10:00,0.316,0.046,242.10,1.4,0.0
731852,7/5/2008,22:56:00,1.368,0.172,242.77,5.6,1.0
683821,4/4/2008,14:25:00,1.384,0.080,243.53,5.6,19.0
...,...,...,...,...,...,...,...
588676,29/1/2008,12:40:00,4.194,0.128,238.41,17.6,55.0
1050667,15/12/2008,08:31:00,2.342,0.060,241.17,9.6,18.0
998535,9/11/2008,03:39:00,0.380,0.116,241.07,1.6,0.0
984077,30/10/2008,02:41:00,0.780,0.064,243.94,3.2,1.0


In [ ]:
y_pred = pd.DataFrame(pipeline_loaded.predict(X_test), columns=labels)
y_pred['datetime'] = pd.to_datetime(X_test['date'] + ' ' + X_test['time'])
y_pred

,sub_metering_1,sub_metering_2,sub_metering_3,datetime
0,0.0,1.4,0.6,NaT
1,0.2,1.8,18.0,NaT
2,0.0,0.0,0.0,NaT
3,0.0,0.1,0.9,NaT
4,0.0,0.9,18.1,NaT
...,...,...,...,...
158107,15.0,24.2,15.6,NaT
158108,0.0,0.3,17.7,NaT
158109,0.0,0.0,0.0,NaT
158110,0.0,1.0,0.0,NaT
